In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  StandardScaler,LabelEncoder
import pickle

In [33]:
data =  pd.read_csv("Churn_Modelling.csv")


In [34]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [35]:
data = data.drop(['RowNumber' , 'CustomerId' , 'Surname'],axis = 1)

In [36]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [37]:
#Encoding Categorical Variable
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [38]:
#One Hot Encoding Geographical Column
from sklearn.preprocessing import OneHotEncoder


In [39]:
onehot_encoder_geo = OneHotEncoder(sparse_output=False)

In [40]:
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])

In [41]:
geo_encoder

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [42]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [43]:
geo_encoded_df = pd.DataFrame(geo_encoder,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [44]:
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [45]:
#Combine one hot encoder columns with original data
data = pd.concat([data.drop(['Geography'],axis = 1) , geo_encoded_df],axis=1)

In [46]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [47]:
## Save the Encoder and scaler

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender , file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo , file)

    

In [48]:
### Divide Data into dependant and independant
X = data.drop(['Exited'] , axis=1)
Y = data['Exited']

X_train , X_test , Y_train , Y_test = train_test_split(X , Y,test_size=0.2,random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [49]:
with open('scaler.pkl' , 'wb') as file:
    pickle.dump(scaler, file)

In [50]:
import tensorflow as tf

In [52]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

In [58]:
(X_train.shape[1],)

(12,)

In [62]:
model = Sequential([
    Dense(64 , activation = 'relu' , input_shape =(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid')
])

In [63]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 64)                832       
                                                                 
 dense_5 (Dense)             (None, 32)                2080      
                                                                 
 dense_6 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [64]:
opt = tf.keras.optimizers.Adam(learning_rate = 0.01)

In [65]:
model.compile(optimizer=opt,loss="binary_crossentropy",metrics =['accuracy'])

In [68]:
from datetime import datetime

In [70]:
### Setup the Tensorboard
log_dir = "logs/fit" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_calback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [75]:
#Setup Early Stoppig
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [76]:
history = model.fit(
    X_train,Y_train,validation_data = (X_test,Y_test),epochs = 100,
    callbacks=[tensorflow_calback,early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3480 - accuracy: 0.8553 - val_loss: 0.3481 - val_accuracy: 0.8570
Epoch 2/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3426 - accuracy: 0.8604 - val_loss: 0.3380 - val_accuracy: 0.8555
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3439 - accuracy: 0.8583 - val_loss: 0.3533 - val_accuracy: 0.8575
Epoch 4/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3376 - accuracy: 0.8626 - val_loss: 0.3380 - val_accuracy: 0.8580
Epoch 5/100
250/250 [==============================] - 2s 10ms/step - loss: 0.3336 - accuracy: 0.8651 - val_loss: 0.3466 - val_accuracy: 0.8620
Epoch 6/100
250/250 [==============================] - 2s 8ms/step - loss: 0.3317 - accuracy: 0.8616 - val_loss: 0.3355 - val_accuracy: 0.8590
Epoch 7/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3323 - accuracy: 0.8635 - val_loss: 0.3558 - val_accuracy: 0.859

In [77]:
model.save('model.h5')

d:\textbook\VAIBHAV DOCUMENTS\DeepLearning\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [85]:
%reload_ext tensorboard

In [86]:
%tensorboard --logdir logs/fit20241003-150152

Reusing TensorBoard on port 6008 (pid 11580), started 0:00:54 ago. (Use '!kill 11580' to kill it.)